In [2]:
import pandas as pd
import numpy as np
import os


In [7]:
price_filename = "/home/ubuntu2010/바탕화면/famafrench_data/stockdata/label_data/USA/x/stock_2d_actor.csv"
directory = "/home/ubuntu2010/바탕화면/famafrench_data/stockdata/label_data/USA"

def make_price_label(filedir):
    data = pd.read_csv(filedir, index_col='date', parse_dates=['date'])

    close_df = data[[i for i in data.columns if 'close' in i]]
    y_data = close_df.shift(-1).fillna(method='ffill').fillna(method='bfill')
    data_logr_whole= np.log(data/data.shift(1).fillna(method='bfill'))
    data_logr_whole = data_logr_whole.fillna(method='ffill')
    data_log_close = np.log(close_df/close_df.shift(1).fillna(method='bfill'))
    data_log_close = data_log_close.fillna(method='ffill')
    data_logr_y = data_log_close.shift(-1).fillna(method='ffill').fillna(method='bfill')

    print("# of Null data: ", close_df.isna().sum().sum()) # no null
    print("# of Null data: ", y_data.isna().sum().sum()) # no null
    print("# of Null data: ", data_logr_whole.isna().sum().sum()) # no null
    print("# of Null data: ", data_log_close.isna().sum().sum()) # no null

    # file save
    # origin price data
    close_df.to_csv("/home/ubuntu2010/바탕화면/famafrench_data/stockdata/label_data/USA/x/stock_2d_onlyclose.csv")
    y_data.to_csv("/home/ubuntu2010/바탕화면/famafrench_data/stockdata/label_data/USA/y/stock_2d_origin_price.csv")

    # log returns data 
    data_logr_whole.to_csv("/home/ubuntu2010/바탕화면/famafrench_data/stockdata/label_data/USA/x/stock_2d_log_all.csv")
    data_log_close.to_csv("/home/ubuntu2010/바탕화면/famafrench_data/stockdata/label_data/USA/x/stock_2d_log_close.csv")
    data_logr_y.to_csv("/home/ubuntu2010/바탕화면/famafrench_data/stockdata/label_data/USA/y/stock_2d_logr_y.csv")
    
    return close_df, y_data, data_logr_whole, data_log_close, data_logr_y



def make_pretrain_dataset(filedir, directory):
    # 
    pretrain_datadir = directory+f"/pretrain"
    os.makedirs(pretrain_datadir, exist_ok=True)
    close_df, y_data, data_logr_whole, data_log_close, data_logr_y = make_price_label(filedir)
    data_dict = {"close":close_df, "log_p":data_log_close, "label":y_data, "log_label":data_logr_y}
    tickers = [i.split("_")[-1] for i in y_data.columns]
    print("tickers: ", tickers)
    # container = {}
    
    
    for tic in tickers:
        container = []
        for col_name, df in data_dict.items():
            print(col_name)
            for col in df.columns:
                if tic == col.split("_")[-1]: container.append(df[col])
        print(tic)
        final_df = pd.DataFrame(container)
        final_df = final_df.transpose()
        final_df.columns = ["close", "log_p", "label", "log_label"]
        print(final_df.head(3))

        final_df.to_csv(pretrain_datadir+f"/{tic}_pretrain_dataset.csv")

    return final_df


make_pretrain_dataset(filedir=price_filename, directory=directory)

# of Null data:  0
# of Null data:  0
# of Null data:  0
# of Null data:  0
tickers:  ['ALK', 'BAX', 'BFb', 'BK', 'BRO', 'CMI', 'CMS', 'CVS', 'CVX', 'DIS', 'EIX', 'FMC', 'IBM', 'JPM', 'K', 'LUMN', 'MMM', 'MSI', 'NEE', 'NKE', 'OMC', 'OXY', 'PHM', 'PPG', 'SHW', 'TYL', 'UHS', 'VFC', 'VLO', 'WMT']
close
log_p
label
log_label
ALK
              close     log_p    label  log_label
date                                             
2000-01-03  8.75000  0.000000  8.64062  -0.012579
2000-01-04  8.64062 -0.012579  8.68750   0.005411
2000-01-05  8.68750  0.005411  8.69531   0.000899
close
log_p
label
log_label
BAX
               close     log_p     label  log_label
date                                               
2000-01-03  16.66893  0.000000  16.29549  -0.022658
2000-01-04  16.29549 -0.022658  17.00842   0.042820
2000-01-05  17.00842  0.042820  17.36488   0.020741
close
log_p
label
log_label
BFb
              close     log_p    label  log_label
date                                             

,close,log_p,label,log_label
date,,,,
2000-01-03,66.8125,0.000000,64.3125,-0.038136
2000-01-04,64.3125,-0.038136,63.0000,-0.020619
2000-01-05,63.0000,-0.020619,63.6875,0.010854
2000-01-06,63.6875,0.010854,68.5000,0.072845
2000-01-07,68.5000,0.072845,67.2500,-0.018417
...,...,...,...,...
2022-02-22,136.4500,-0.011223,135.0500,-0.010313
2022-02-23,135.0500,-0.010313,134.5300,-0.003858
2022-02-24,134.5300,-0.003858,136.3800,0.013658
